# Thicket Nsight Compute Reader: Thicket Tutorial

Nsight Compute (NCU) is a performance profiler for NVIDIA GPUs. NCU report files do not have a calltree, but with the NVTX Caliper service we can forward Caliper annotations to NCU. By profiling the same executable with a calltree profiler like Caliper, we can map the NCU data to the calltree profile and create a Thicket object. 

In Section 6, we reproduce some of the analysis and visualizations from the paper:

Olga Pearce, Jason Burmark, Rich Hornung, Befikir Bogale, Ian Lumsden, Michael McKinsey, Dewi Yokelson, David Boehme, Stephanie Brink, Michela Taufer, and Tom Scogland. “RAJA Performance Suite: Performance Portability Analysis with Caliper and Thicket”. SC-W 2024: Workshops of ACM/IEEE International Conference for High Performance Computing, Networking, Storage, and Analysis. Performance, Portability & Productivity in HPC. 2024.

***

## 1. Import Necessary Packages

The Thicket NCU reader requires an existing install of Nsight Compute, and the `extras/python` directory in the Nsight Compute installation directory must be added to the `PYTHONPATH`. We use `sys.path.append` to add the path to the `PYTHONPATH` in this notebook. If you are not on a Livermore Computing system, you must change this path to match your install of Nsight Compute.

**VERSION NOTICE: This functionality is tested with nsight-compute version 2023.2.2. Your mileage may vary if using a different version.**

In [ ]:
import os
import sys

sys.path.append("/usr/tce/packages/nsight-compute/nsight-compute-2023.2.2/extras/python")

from IPython.display import display
from IPython.display import HTML

import thicket as tt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

display(HTML("<style>.container { width:80% !important; }</style>"))

## 2. The Dataset

The dataset we are using comes from a profile of the RAJA Performance Suite on Lassen. We profile the `block_128` tuning of the `Base_CUDA`, `Lambda_CUDA`, and `RAJA_CUDA` variants, while varying the problem size for 1 million and 2 million. The calltree profiles come from the CUDA Activity Profile Caliper configuration. By changing the `variant` argument in the following cell, we can look at NCU data for different variants.

The following are reproducible steps to generate this dataset:

```
# Example of building
$ . RAJAPerf/scripts/lc-builds/blueos_nvhpc_nvcc_clang_caliper.sh 
$ make -j

# Load CUDA version equal to the CUDA version used to build RAJAPerf
$ module load nvhpc/24.1-cuda-11.2.0

# Turn off NVIDIA Data Center GPU Manager (DCGM) on Lassen so we can run NCU (get an error if it's on)
$ dcgmi profile --pause
```

```
# Example run to Generate the CUDA Activity Profile
$ CALI_CONFIG=cuda-activity-profile,output.format=cali lrun -n 1 --smpiargs="-disable_gpu_hooks" bin/raja-perf.exe --variants [Base_CUDA OR Lambda_CUDA OR RAJA_CUDA] --tunings block_128 --size [1048576 OR 2097152] --repfact 0.01

# Example run to Generate the NCU Report
$ CALI_SERVICES_ENABLE=nvtx lrun -n 1 --smpiargs="-disable_gpu_hooks" ncu \
--nvtx --set default \
--export report \
--metrics sm__throughput.avg.pct_of_peak_sustained_elapsed \
--replay-mode application \
bin/raja-perf.exe --variants [Base_CUDA OR Lambda_CUDA OR RAJA_CUDA] --tunings block_128 --size [1048576 OR 2097152] --repfact 0.01
```

In [2]:
# Map all files
ncu_dir = "../data/ncu/"
ncu_report_mapping = {}
variant = "base_cuda" # OR "lambda_cuda" OR "raja_cuda"
problem_sizes = ["1M", "2M"]
for problem_size in problem_sizes:
    full_path = f"{ncu_dir}{variant}/{problem_size}/"
    ncu_report_mapping[full_path + "report.ncu-rep"] = full_path + "cuda_profile.cali"

## 3. Read Calltree Profiles into Thicket

The only performance metrics contained in the CUDA Activity Profile will be the CPU time `time` and the GPU time `time (gpu)`.

In [ ]:
tk_cap = tt.Thicket.from_caliperreader(list(ncu_report_mapping.values()))
tk_cap.dataframe.head(20)

## 4. Add NCU Data

The Thicket `add_ncu` function takes one required argument and one optional arguement. The required argument `ncu_report_mapping` is the mapping from the NCU report file to the corresponding calltree profile run. The optional argument `chosen_metrics` allows for a subselection of the NCU performance metrics to add, since there can be hundreds of NCU performance metrics. By default we add all metrics.

In [ ]:
# Add NCU to thicket
ncu_metrics = [
    "gpu__time_duration.sum",
    "sm__throughput.avg.pct_of_peak_sustained_elapsed",
    "smsp__maximum_warps_avg_per_active_cycle",
]
# Add in metrics
tk_cap.add_ncu(
    ncu_report_mapping=ncu_report_mapping, 
    chosen_metrics=ncu_metrics,
)
tk_cap.dataframe.head(20)

## 5. Visualize the NCU Performance Data on the Calltree

In [ ]:
print(tk_cap.tree(
    metric_column="sm__throughput.avg.pct_of_peak_sustained_elapsed",
    expand_name=True,
    ))

## 6. Create Instruction Roofline Plots

We can make roofline plots using the metrics we have collected using Nsight Compute. The Roofline sets an upper bound on performance of a kernel depending on its operational intensity. We will use roofline plots to understand the performance of RAJAPerf kernels.

In this section, we reproduce some of the analysis and visualizations from the paper:

Olga Pearce, Jason Burmark, Rich Hornung, Befikir Bogale, Ian Lumsden, Michael McKinsey, Dewi Yokelson, David Boehme, Stephanie Brink, Michela Taufer, and Tom Scogland. “RAJA Performance Suite: Performance Portability Analysis with Caliper and Thicket”. SC-W 2024: Workshops of ACM/IEEE International Conference for High Performance Computing, Networking, Storage, and Analysis. Performance, Portability & Productivity in HPC. 2024.

Instruction Roofline Models were introduced by Ding et. al to better characterize GPU workloads by looking at instruction intensity. We walk through the creation of instruction roofline models for the Appications group of the RAJAPerf kernels below. 

More references on methodology for roofline models:

* Samuel Williams, Andrew Waterman, and David Patterson. 2009. Roofline: an insightful visual performance model for multicore architectures. Commun. ACM 52, 4 (April 2009), 65–76. https://doi.org/10.1145/1498765.1498785

* Nan Ding, Muaaz Awan, Samuel Williams. 2022. Instruction Roofline: An insightful visual performance model for GPUs. Concurrency and Computation: Practice and Experience
https://doi.org/10.1002/cpe.6591

Step 1 is to create a Thicket using a CUDA Activity Profile, and add the NCU data. We use `RAJA_CUDA` variant data of the `block_256` tuning.

Command to generate the Caliper CUDA Activity Profile:
```
CALI_CONFIG=cuda-activity-profile,output.format=cali lrun -n 1 --smpiargs="-disable_gpu_hooks" bin/raja-perf.exe --variants RAJA_CUDA --tunings block_256 --size 8388608
```

Command to generate the NCU report with Roofline metrics:
```
CALI_SERVICES_ENABLE=nvtx lrun -n 1 --smpiargs="-disable_gpu_hooks" ncu \
--nvtx --set default \
--export report \
--metrics sm__sass_thread_inst_executed.sum,l1tex__t_sectors_pipe_lsu_mem_global_op_ld.sum,l1tex__t_sectors_pipe_lsu_mem_global_op_st.sum,l1tex__t_sectors_pipe_lsu_mem_local_op_ld.sum,l1tex__t_sectors_pipe_lsu_mem_local_op_st.sum,lts__t_sectors_op_read.sum,lts__t_sectors_op_write.sum,lts__t_sectors_op_atom.sum,lts__t_sectors_op_red.sum,dram__sectors_read.sum,dram__sectors_write.sum,l1tex__data_pipe_lsu_wavefronts_mem_shared_op_ld.sum,l1tex__data_pipe_lsu_wavefronts_mem_shared_op_st.sum \
--replay-mode application \
bin/raja-perf.exe --variants RAJA_CUDA --tunings block_256 --size 8388608
```

In [ ]:
roofline_cali = ncu_dir + "roofline/block_256_profile.cali"
roofline_ncu = ncu_dir + "roofline/block_256_profile.ncu-rep"

# Unzip files, unless they have already been unzipped
if not (os.path.isfile(roofline_cali) and os.path.isfile(roofline_ncu)):
    import zipfile
    with zipfile.ZipFile(ncu_dir + "roofline/block_256_profile.zip", "r") as zip_ref:
        zip_ref.extractall(ncu_dir + "roofline/")

# Create Thicket and add NCU data
tk_roofline = tt.Thicket.from_caliperreader(roofline_cali)
tk_roofline.add_ncu({roofline_ncu:roofline_cali})

Select a subset of kernels to plot on the roofline. Here we are just selecting `Apps` kernels.

In [47]:
# Options (can be more than one): ["Algorithm", "Apps", "Basic", "Comm", "Lcals", "Polybench", "Stream"]
kernel_types = ["Apps"]

raja_kernel_query = (
    tt.query.Query()
    .match (
        ".",
        lambda row: row["name"].apply(
            lambda x: any([x.startswith(c + "_") for c in kernel_types])
        ).all()
    )
    .rel("*")
)

pruned_th = tk_roofline.query(raja_kernel_query)

Aggregate metrics for kernels with multiple instances

In [ ]:
# Match parent kernel name, e.g. "Apps_VOL3D"
kernel_query = """
MATCH (".",p)->("*")
WHERE p."name" = "{ker}"
"""

# Match demangled kernel that contains NCU measurement.
child_kernel_query = """
MATCH (".", p)
WHERE p."depth" = 2
"""

# List of columns to exclude from aggregation
cols = [col for col in pruned_th.dataframe.columns.tolist() if col not in ["nid", "time", "name", "time (inc)", "time (gpu) (inc)"]]
leaves = []

for n in pruned_th.graph.roots:
    kernels = {}
    
    kernels["name"] = n.frame.get("name")

    tmp = kernel_query.format(ker=n.frame.get("name"))
    ker_th = pruned_th.query(tmp, multi_index_mode="all")
    leaf = ker_th.query(child_kernel_query, multi_index_mode="all")
    for col in cols:
        kernels[col] = leaf.dataframe[col].sum()
    
    leaves.append(kernels)

agg_df = pd.DataFrame(data=leaves)
agg_df

Calculate instruciton intensities from existing NCU metrics.

In [49]:
# Warp Instructions
agg_df["Warp Instructions"] = agg_df["sm__sass_thread_inst_executed.sum"] / 32
# L1 Global Count Memory Transactions
agg_df["L1 (GLOBAL)"] = agg_df["l1tex__t_sectors_pipe_lsu_mem_global_op_ld.sum"] + agg_df["l1tex__t_sectors_pipe_lsu_mem_global_op_st.sum"]
agg_df["L1 (SHARED)"] = agg_df["l1tex__data_pipe_lsu_wavefronts_mem_shared_op_ld.sum"] + agg_df["l1tex__data_pipe_lsu_wavefronts_mem_shared_op_st.sum"]
agg_df["Total L1 Transactions"] = agg_df["L1 (GLOBAL)"] + (4 * agg_df["L1 (SHARED)"])
# Shared Count Memory Transactions
agg_df["L2 Write Transactions"] =  agg_df["lts__t_sectors_op_write.sum"] + agg_df["lts__t_sectors_op_atom.sum"] + agg_df["lts__t_sectors_op_red.sum"]
agg_df["L2 Read Transactions"] =  agg_df["lts__t_sectors_op_read.sum"] + agg_df["lts__t_sectors_op_atom.sum"] + agg_df["lts__t_sectors_op_red.sum"]
agg_df["Total L2 Transactions"] = agg_df["L2 Read Transactions"] + agg_df["L2 Write Transactions"]
# HBM Count Memory Transactions
agg_df["HBM Transactions"] = agg_df["dram__sectors_read.sum"] +  agg_df["dram__sectors_write.sum"]
# L1, L2, HBM Intensities
agg_df["L1 Instruction Intensity"] = agg_df["Warp Instructions"] / agg_df["Total L1 Transactions"]
agg_df["L2 Instruction Intensity"] = agg_df["Warp Instructions"] / agg_df["Total L2 Transactions"]
agg_df["HBM Instruction Intensity"] = agg_df["Warp Instructions"] / agg_df["HBM Transactions"]
# Performance in GIPS
agg_df["Performance GIPS"] = agg_df["Warp Instructions"] / (agg_df["time (gpu)"] * (10 ** 9))

### Plotting Roofline with Matplotlib

The plotting code below was adapted from the following resource: https://gitlab.com/NERSC/roofline-on-nvidia-gpus/-/blob/master/custom-scripts/roofline.py

In [50]:
pruned_th_kers = tk_roofline.query(child_kernel_query, multi_index_mode="all")
metrics = ["L1 Instruction Intensity", "L2 Instruction Intensity", "HBM Instruction Intensity"]

c_s = ["red", "blue", "green", "orange", "purple", "cyan", "magenta"]
final_colors = []
for i in agg_df["name"].tolist():
    for j in range(0, len(kernel_types)):
        if i.startswith(kernel_types[j]):
            final_colors.append(c_s[j])
            continue

font = {"size": 15}
plt.rc("font", **font)

colors = ["tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:gray", "tab:olive", "tab:cyan"]
styles = ["o", "s", "v", "^", "D", ">", "<", "*", "h", "H", "+", "1", "2", "3", "4", "8", "p", "d", "|", "_", ".", ","]

markersize = 10
markerwidth = 2
maxchar = 25

def roofline(LABELS, flag="HBM", data_df=None):
    LABELS = [x[:maxchar] for x in LABELS]
    
    bandiwdth_hbm = 25.9 # in GTXN/s
    bandwidth_l2 = 93.6 # in GTXN/s
    bandwidth_l1 = 437.5 # in GTXN/s
    
    if flag == "L1":
        memRoofs = [("L1", bandwidth_l1)]
    elif flag == "L2":
        memRoofs = [("L2", bandwidth_l2)]
    elif flag == "HBM":
        memRoofs = [("HBM", bandiwdth_hbm)]
    elif flag == "all":
        memRoofs = [("L1", bandwidth_l1), ("L2", bandwidth_l2),  ("HBM", bandiwdth_hbm)]

    cmpRoofs = [("GIPS", 489.6)]

    fig = plt.figure(1, figsize=(10,6))
    plt.clf()
    ax = fig.gca()
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel("Instruction Intensity [Warp Instructions/transaction]")
    ax.set_ylabel("Performance [Warp GIPS]")

    nx   = 10000
    xmin = -3 
    xmax = 3
    ymin = 1
    ymax = 1000

    ax.set_xlim(10 ** xmin, 10 ** xmax)
    ax.set_ylim(ymin, ymax)

    ixx = int(nx * 0.02)
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    scomp_x_elbow  = []
    scomp_ix_elbow = []
    smem_x_elbow   = []
    smem_ix_elbow  = []

    x = np.logspace(xmin, xmax, nx)
    for roof in cmpRoofs:
        for ix in range(1, nx):
            if float(memRoofs[0][1] * x[ix]) >= roof[1] and (memRoofs[0][1] * x[ix - 1]) < roof[1]:
                scomp_x_elbow.append(x[ix - 1])
                scomp_ix_elbow.append(ix - 1)
                break

    for roof in memRoofs:
        for ix in range(1, nx):
            if (cmpRoofs[0][1] <= roof[1] * x[ix] and cmpRoofs[0][1] > roof[1] * x[ix - 1]):
                smem_x_elbow.append(x[ix - 1])
                smem_ix_elbow.append(ix - 1)
                break

    for i in range(len(cmpRoofs)):
        roof = cmpRoofs[i][1]
        y = np.ones(len(x)) * roof
        ax.plot(x[scomp_ix_elbow[i]:], y[scomp_ix_elbow[i]:], c="k", ls="-", lw="2")

    for i in range(len(memRoofs)):
        roof = memRoofs[i][1]
        y = x * roof
        ax.plot(x[:smem_ix_elbow[i] + 1], y[:smem_ix_elbow[i] + 1], c="k", ls="-", lw="2")

    for roof in cmpRoofs:
        ax.text(
            x[-ixx],
            roof[1],
            roof[0] + ": " + "{0:.1f}".format(roof[1]) + " Warp GIPS",
            horizontalalignment="right",
            verticalalignment="bottom"
        )

    for roof in memRoofs:
        ang = np.arctan(np.log10(xlim[1] / xlim[0]) / np.log10(ylim[1] / ylim[0])
                                   * fig.get_size_inches()[1] / fig.get_size_inches()[0] )
        if x[ixx]*roof[1] > ymin:
            ax.text(
                x[ixx],
                x[ixx] * roof[1] * (1 + 0.25 * np.sin(ang) ** 2),
                roof[0] + ": " + "{0:.1f}".format(float(roof[1])) + " GTXN/s",
                horizontalalignment="left",
                verticalalignment="bottom",
                rotation=180 / np.pi * ang
            )
        else:
            ymin_ix_elbow=list()
            ymin_x_elbow=list()
            for ix in range(1, nx):
                if (ymin <= roof[1] * x[ix] and ymin > roof[1] * x[ix - 1]):
                    ymin_x_elbow.append(x[ix - 1])
                    ymin_ix_elbow.append(ix - 1)
                    break
            ax.text(
                x[ixx+ymin_ix_elbow[0]],
                x[ixx+ymin_ix_elbow[0]] * roof[1] * (1 + 0.25 * np.sin(ang) ** 2) * 1.15,
                roof[0] + ": " + "{0:.1f}".format(float(roof[1])) + " GTXN/s",
                horizontalalignment="left",
                verticalalignment="bottom",
                rotation=180 / np.pi * ang
            )

    if flag == "L1":
        ax.scatter(data_df[metrics[0]], data_df["Performance GIPS"], c=final_colors, label="L1", marker=styles[0])
    elif flag == "L2":
        ax.scatter(data_df[metrics[1]], data_df["Performance GIPS"], c=final_colors, label="L2", marker=styles[1])
    elif flag == "HBM":
        ax.scatter(data_df[metrics[2]], data_df["Performance GIPS"], c=final_colors, label="HBM", marker="*")

    elif flag == "all":
        ax.scatter(data_df[metrics[0]], data_df["Performance GIPS"], c=final_colors, label="L1", marker=styles[0])
        ax.scatter(data_df[metrics[1]], data_df["Performance GIPS"], c=final_colors, label="L2", marker=styles[1])
        ax.scatter(data_df[metrics[2]], data_df["Performance GIPS"], c=final_colors, label="HBM", marker="*")

    custom_labels = [k.split("_")[0] for k in kernel_types]
    custom_handles = [plt.Line2D([0], [0], color=i, lw=2) for i in c_s]
    
    if flag == "HBM":
        leg2 = ax.legend(
            custom_handles,
            custom_labels,
            bbox_to_anchor=(0.36, 1),
            title="Kernel Types",
            fontsize="12",          # Adjust label font size (e.g., "small", "medium", "large", or specific size)
            title_fontsize="small"
        )
    else:
        leg2 = ax.legend(
            custom_handles,
            custom_labels,
            bbox_to_anchor=(0.36, 1),
            title="Kernel Types",
            fontsize="12",          # Adjust label font size (e.g., "small", "medium", "large", or specific size)
            title_fontsize="small"
        )
        
    ax.add_artist(leg2)
    
    ax.legend(loc="upper left")
    plt.show()

In [ ]:
roofline(LABELS=pruned_th.dataframe["name"].tolist(), flag="all", data_df=agg_df)

In [ ]:
roofline(LABELS=pruned_th.dataframe["name"].tolist(), flag="L1", data_df=agg_df)

In [ ]:
roofline(LABELS=pruned_th.dataframe["name"].tolist(), flag="L2", data_df=agg_df)

In [ ]:
roofline(LABELS=pruned_th.dataframe["name"].tolist(), flag="HBM", data_df=agg_df)